In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("F1") \
    .getOrCreate()

25/05/05 20:28:24 WARN Utils: Your hostname, bdlc-004 resolves to a loopback address: 127.0.1.1; using 10.176.129.6 instead (on interface ens192)
25/05/05 20:28:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/05 20:28:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/05 20:28:25 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/05/05 20:28:25 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


# Download Datasets

1. Create the required directories:

In [2]:
!hdfs dfs -rm -r -f /f1/raw
!hdfs dfs -mkdir -p /f1/raw
!mkdir -p /data/dataset_cluster/f1

Deleted /f1/raw


2. Download the following Datasets as ZIP:
   - https://www.kaggle.com/datasets/rohanrao/formula-1-world-championship-1950-2020?resource=download
   - https://www.kaggle.com/datasets/quantumkaze/f1-weather-dataset-2018-2023/data?select=F1+Weather%282023-2018%29.csv
4. Move the ZIP files to:
   - `/data/dataset_cluster/f1/f1.zip`
   - `/data/dataset_cluster/f1/weather.zip`
6. Unzip the files and move the contents to HDFS

In [8]:
!(cd /data/dataset_cluster/f1; unzip -o f1.zip; unzip -o weather.zip)

Archive:  f1.zip
  inflating: circuits.csv            
  inflating: constructor_results.csv  
  inflating: constructor_standings.csv  
  inflating: constructors.csv        
  inflating: driver_standings.csv    
  inflating: drivers.csv             
  inflating: lap_times.csv           
  inflating: pit_stops.csv           
  inflating: qualifying.csv          
  inflating: races.csv               
  inflating: results.csv             
  inflating: seasons.csv             
  inflating: sprint_results.csv      
  inflating: status.csv              
Archive:  weather.zip
  inflating: F1 Weather(2023-2018).csv  


In [10]:
!(cd /data/dataset_cluster/f1; mv ./F1\ Weather*.csv weather.csv; ls *.csv)

mv: cannot stat './F1 Weather*.csv': No such file or directory
circuits.csv		   driver_standings.csv  results.csv
constructor_results.csv    lap_times.csv	 seasons.csv
constructors.csv	   pit_stops.csv	 sprint_results.csv
constructor_standings.csv  qualifying.csv	 status.csv
drivers.csv		   races.csv		 weather.csv


In [11]:
!hdfs dfs -Ddfs.replication=1 -put -f /data/dataset_cluster/f1/*.csv /f1/raw/

In [18]:
!hdfs dfs -ls /f1/raw

Found 15 items
-rw-r--r--   1 cluster supergroup      10104 2025-05-05 20:30 /f1/raw/circuits.csv
-rw-r--r--   1 cluster supergroup     219365 2025-05-05 20:30 /f1/raw/constructor_results.csv
-rw-r--r--   1 cluster supergroup     317206 2025-05-05 20:30 /f1/raw/constructor_standings.csv
-rw-r--r--   1 cluster supergroup      17478 2025-05-05 20:30 /f1/raw/constructors.csv
-rw-r--r--   1 cluster supergroup     883771 2025-05-05 20:30 /f1/raw/driver_standings.csv
-rw-r--r--   1 cluster supergroup      94367 2025-05-05 20:30 /f1/raw/drivers.csv
-rw-r--r--   1 cluster supergroup   17622395 2025-05-05 20:30 /f1/raw/lap_times.csv
-rw-r--r--   1 cluster supergroup     443719 2025-05-05 20:30 /f1/raw/pit_stops.csv
-rw-r--r--   1 cluster supergroup     465231 2025-05-05 20:30 /f1/raw/qualifying.csv
-rw-r--r--   1 cluster supergroup     164344 2025-05-05 20:30 /f1/raw/races.csv
-rw-r--r--   1 cluster supergroup    1721961 2025-05-05 20:30 /f1/raw/results.csv
-rw-r--r--   1 cluster supergroup    

# Convert CSVs to Parquet

In [13]:
!hdfs dfs -mkdir -p /f1/data

In [14]:
hadoop = spark._jvm.org.apache.hadoop
fs = hadoop.fs.FileSystem
conf = hadoop.conf.Configuration()
path = hadoop.fs.Path('/f1/raw')
for f in fs.get(conf).listStatus(path):
    if not f.getLocalName().endswith(".csv"): continue
    parquet_filename = f.getLocalName().replace(".csv", ".parquet")
    df = spark.read.csv(f"/f1/raw/{f.getLocalName()}", header=True, inferSchema=True)
    df.write.mode('overwrite').parquet(f"/f1/data/{parquet_filename}")

In [20]:
!hdfs dfs -ls -h /f1/raw

Found 15 items
-rw-r--r--   1 cluster supergroup      9.9 K 2025-05-05 20:30 /f1/raw/circuits.csv
-rw-r--r--   1 cluster supergroup    214.2 K 2025-05-05 20:30 /f1/raw/constructor_results.csv
-rw-r--r--   1 cluster supergroup    309.8 K 2025-05-05 20:30 /f1/raw/constructor_standings.csv
-rw-r--r--   1 cluster supergroup     17.1 K 2025-05-05 20:30 /f1/raw/constructors.csv
-rw-r--r--   1 cluster supergroup    863.1 K 2025-05-05 20:30 /f1/raw/driver_standings.csv
-rw-r--r--   1 cluster supergroup     92.2 K 2025-05-05 20:30 /f1/raw/drivers.csv
-rw-r--r--   1 cluster supergroup     16.8 M 2025-05-05 20:30 /f1/raw/lap_times.csv
-rw-r--r--   1 cluster supergroup    433.3 K 2025-05-05 20:30 /f1/raw/pit_stops.csv
-rw-r--r--   1 cluster supergroup    454.3 K 2025-05-05 20:30 /f1/raw/qualifying.csv
-rw-r--r--   1 cluster supergroup    160.5 K 2025-05-05 20:30 /f1/raw/races.csv
-rw-r--r--   1 cluster supergroup      1.6 M 2025-05-05 20:30 /f1/raw/results.csv
-rw-r--r--   1 cluster supergroup    

In [19]:
!hdfs dfs -ls -h /f1/data/**

Found 2 items
-rw-r--r--   2 cluster supergroup          0 2025-05-05 20:30 /f1/data/circuits.parquet/_SUCCESS
-rw-r--r--   2 cluster supergroup      9.0 K 2025-05-05 20:30 /f1/data/circuits.parquet/part-00000-ecc1be5b-1e26-44c6-9eb7-ef1797689e01-c000.snappy.parquet
Found 2 items
-rw-r--r--   2 cluster supergroup          0 2025-05-28 21:56 /f1/data/cleaned_circuits.parquet/_SUCCESS
-rw-r--r--   2 cluster supergroup      3.3 K 2025-05-28 21:56 /f1/data/cleaned_circuits.parquet/part-00000-fb3403c2-e9eb-42c6-a1a1-d4bc3c225313-c000.snappy.parquet
Found 2 items
-rw-r--r--   2 cluster supergroup          0 2025-04-27 17:30 /f1/data/cleaned_constructor_results.parquet/_SUCCESS
-rw-r--r--   2 cluster supergroup     78.8 K 2025-04-27 17:30 /f1/data/cleaned_constructor_results.parquet/part-00000-23998559-62d1-4e57-baa4-d4d192fefdfc-c000.snappy.parquet
Found 2 items
-rw-r--r--   2 cluster supergroup          0 2025-05-28 20:24 /f1/data/cleaned_constructor_standings.parquet/_SUCCESS
-rw-r--r--   